<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.6

In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers
from sklearn.datasets import load_breast_cancer, load_boston

ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#Dataframes
Wikipedia = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Wikipedia.csv') #text
Twitter = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Twitter.csv') #text

#Series
articles = Wikipedia['Text']
labels = Wikipedia['AI']
tweets = Twitter['tweet']
sentiments = Twitter['is_there_an_emotion_directed_at_a_brand_or_product']

#arrays
cancer_data = load_breast_cancer()['data']
diagnosis = dl.MulticlassOutput(load_breast_cancer()['target'])
housing_data = load_boston()['data']
prices = load_boston()['target']

#images
#uploaded = files.upload()
#data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
#data.extractall()

In [ ]:
#image directories... if any

# Parameter Grid Templates

rnn_grid = {'output_dim': [int], 'embedding': [vocab], 'nodes': [int], 'activation': [str], 
              'regularizer': [None or str], 'stacking': [bool], 'dropout': [bool], 'optimizer': [str], 
              'method': [str], 'bidirectional': [bool]} 

cnn_grid = {'output_dim': [int], 'base_filters':[int], 'kernel_size': [int], 'activation': [str], 
            'nblocks': [int], 'pool':[int], 'dropout': [bool], 'closer': [bool], 'optimizer': [str], 
            'metrics': [str]}

tc_grid = {'output_dim': [int], 'nodes': [int], 'activation': [str], 'regularizer': [None or str], 'stacking': [bool], 
               'dropout': [bool], 'nlayers': [int], 'closer': [bool], 'loss': [str], 'optimizer': [str]}

tr_grid = {'nodes': [int], 'activation': [str], 'regularizer': [None or str], 'stacking': [bool], 'dropout': [bool], 'nlayers': [int], 'closer': [bool], 
                'loss': [str], 'optimizer': [str], 'y_col': [str]}

In [60]:
def TCP(X, y):
  dl = DeepLearning()
  avg = 0
  words = []
  for i in range(len(X)):
    avg += len(list(X)[i].split(' '))/len(X)
    word_lst = list(X)[i].replace('\n', ' ').split(' ')
    for word in word_lst:
      words.append(word.replace('.', '').replace(',', '').replace(' ', '').replace('"', '').replace(':', '').replace(';', '').replace('!', ''))
  pad = int(avg)
  text, labels = dl.ModelReadyText1(X, y, pad)    
  return text, labels, len(np.unique(words))

In [43]:
def Preprocess(df, data_str, target_str):
  '''fully preprocesses a dataframe of Text Documents for classification'''
  avg = 0
  for i in range(len(df[data_str])):
    avg += len(list(df[data_str])[i].split(' '))/len(df)
  pad = int(avg)
  text, labels = dl.ModelReadyText1(df[data_str], df[target_str], pad)
  return text, labels

blogs, blog_labels = Preprocess(Wikipedia, 'Text', 'AI')
tweets, tweet_labels = Preprocess(Twitter, 'tweet', 'is_there_an_emotion_directed_at_a_brand_or_product')

In [44]:
def GetVocab(df, data_str):
  words = []
  for i in range(len(df)):
    word_lst = list(df[data_str])[i].replace('\n', ' ').split(' ')
    for word in word_lst:
      words.append(word.replace('.', '').replace(',', '').replace(' ', '').replace('"', '').replace(':', '').replace(';', '').replace('!', ''))
  return len(np.unique(words))

In [46]:
blogs_vocab = GetVocab(Wikipedia, 'Text')
tweets_vocab = GetVocab(Twitter, 'tweet')

In [62]:
def BigDawg(X, y, params, task):
  dl = DeepLearning()
  if task == 'NLP':
    X2, y2, _ = TCP(X, y)
    func = dl.RNN
  try:
    return dl.CollectPerformance(params, func, X2, y2)
  except:
    return dl.CollectPerformance(params, func, X2, y2.reshape(-1))

In [63]:
_, _, embed = TCP(articles, labels)
rnn_grid = {'output_dim': [2], 'embedding': [embed], 'nodes': [64], 'activation': ['tanh', 'relu'], 'regularizer': [None, 'L1'], 
            'stacking': [False], 'dropout': [False], 'optimizer': ['adam'], 'method': ['GRU'], 'bidirectional': [False]}

#X = blogs
#y = blog_labels

#try:
  #print(dl.CollectPerformance(rnn_grid, dl.RNN, X, y))
#except:
  #print(dl.CollectPerformance(rnn_grid, dl.RNN, X, y.reshape(-1)))

BigDawg(articles, labels, rnn_grid, 'NLP')

preforming 4 total fits ...
Epoch 1/50
9/9 [==============================] - 10s 934ms/step - loss: 0.6864 - accuracy: 0.6110 - val_loss: 0.7904 - val_accuracy: 0.0000e+00
Epoch 2/50
9/9 [==============================] - 8s 902ms/step - loss: 0.6493 - accuracy: 0.6571 - val_loss: 0.9369 - val_accuracy: 0.0000e+00
Epoch 3/50
9/9 [==============================] - 8s 850ms/step - loss: 0.6087 - accuracy: 0.6061 - val_loss: 1.0260 - val_accuracy: 0.0000e+00
Epoch 4/50
9/9 [==============================] - 8s 846ms/step - loss: 0.4926 - accuracy: 0.7119 - val_loss: 0.7594 - val_accuracy: 0.5139
Epoch 5/50
9/9 [==============================] - 8s 838ms/step - loss: 0.5642 - accuracy: 0.8532 - val_loss: 3.0099 - val_accuracy: 0.0000e+00
Epoch 6/50
9/9 [==============================] - 8s 849ms/step - loss: 1.1413 - accuracy: 0.6198 - val_loss: 2.4633 - val_accuracy: 0.0000e+00
Epoch 7/50
9/9 [==============================] - 7s 831ms/step - loss: 0.9394 - accuracy: 0.6196 - val_loss: 1

,output_dim,embedding,nodes,activation,regularizer,stacking,dropout,optimizer,bidirectional,epochs,batch_size,method,accuracy,loss,val_accuracy,val_loss
0,2,45589,64,tanh,None,False,False,adam,False,9,32,GRU,0.625000,0.753217,0.000000,1.485980
1,2,45589,64,tanh,L1,False,False,adam,False,10,32,GRU,1.000000,0.025232,0.666667,1.206231
2,2,45589,64,relu,None,False,False,adam,False,12,32,GRU,1.000000,0.055502,0.569444,0.840654
3,2,45589,64,relu,L1,False,False,adam,False,6,32,GRU,0.621528,1.106259,0.000000,2.593630


# Image dataset directory graveyard
copy and paste these into a code cell when working with the dataset

Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']